In [ ]:
import numpy as np

In [ ]:
def general_1d(n):
    # Tabela original para o intervalo [-1, 1]
    gauss_points_original = {
        1: ([0], [2]),
        2: ([np.sqrt(1/3), -np.sqrt(1/3)],
            [1, 1]),
        3: ([0, np.sqrt(3/5), -np.sqrt(3/5)],
            [8/9, 5/9, 5/9]),
        4: (
            [
                +np.sqrt(3/7 - 2/7 * np.sqrt(6/5)),
                -np.sqrt(3/7 - 2/7 * np.sqrt(6/5)),
                +np.sqrt(3/7 + 2/7 * np.sqrt(6/5)),
                -np.sqrt(3/7 + 2/7 * np.sqrt(6/5)),
            ],
            [
                (18 + np.sqrt(30)) / 36,
                (18 + np.sqrt(30)) / 36,
                (18 - np.sqrt(30)) / 36,
                (18 - np.sqrt(30)) / 36,
            ],
        ),
    }

    # Verificar se a ordem 'n' está disponível na tabela
    if n not in gauss_points_original:
        raise ValueError(f"Ordem de quadratura {n} não suportada.")

    # Obter pontos e pesos para a ordem m
    points, weights = gauss_points_original[n]

    # Transformar para o intervalo [0, 1]
    transformed_points = [(1 + p) / 2 for p in points]
    transformed_weights = [w / 2 for w in weights]

    return transformed_points, transformed_weights


# `general_triangle_rule()`

In [ ]:
def general_triangle_rule(m, p):
    # Tabela 4.1: Dados da Quadratura de Gauss para Triângulos
    quadrature_table = {
        (1, 1): [  # m=1, p=1
            {"i": 1, "L1": 0.33333333, "L2": 0.33333333, "L3": 0.33333333, "W": 1.00000000}
        ],
        (3, 2): [  # m=3, p=2
            {"i": 1, "L1": 0.5, "L2": 0.5, "L3": 0.0, "W": 0.33333333},
            {"i": 2, "L1": 0.0, "L2": 0.5, "L3": 0.5, "W": 0.33333333},
            {"i": 3, "L1": 0.5, "L2": 0.0, "L3": 0.5, "W": 0.33333333}
        ],
        (4, 3): [  # m=4, p=3
            {"i": 1, "L1": 0.33333333, "L2": 0.33333333, "L3": 0.33333333, "W": -0.56250000},
            {"i": 2, "L1": 0.73333333, "L2": 0.13333333, "L3": 0.13333333, "W": 0.52083333},
            {"i": 3, "L1": 0.13333333, "L2": 0.73333333, "L3": 0.13333333, "W": 0.52083333},
            {"i": 4, "L1": 0.13333333, "L2": 0.13333333, "L3": 0.73333333, "W": 0.52083333}
        ],
        (7, 4): [  # m=7, p=4
            {"i": 1, "L1": 0.33333333, "L2": 0.33333333, "L3": 0.33333333, "W": 0.45000000},
            {"i": 2, "L1": 0.5, "L2": 0.5, "L3": 0.0, "W": 0.13333333},
            {"i": 3, "L1": 0.0, "L2": 0.5, "L3": 0.5, "W": 0.13333333},
            {"i": 4, "L1": 0.5, "L2": 0.0, "L3": 0.5, "W": 0.13333333},
            {"i": 5, "L1": 1.0, "L2": 0.0, "L3": 0.0, "W": 0.05000000},
            {"i": 6, "L1": 0.0, "L2": 1.0, "L3": 0.0, "W": 0.05000000},
            {"i": 7, "L1": 0.0, "L2": 0.0, "L3": 1.0, "W": 0.05000000}
        ],
        (7, 5): [  # m=7, p=5
            {"i": 1, "L1": 0.33333333, "L2": 0.33333333, "L3": 0.33333333, "W": 0.22500000},
            {"i": 2, "L1": 0.05971587, "L2": 0.47014206, "L3": 0.47014206, "W": 0.13239415},
            {"i": 3, "L1": 0.47014206, "L2": 0.05971587, "L3": 0.47014206, "W": 0.13239415},
            {"i": 4, "L1": 0.47014206, "L2": 0.47014206, "L3": 0.05971587, "W": 0.13239415},
            {"i": 5, "L1": 0.79742699, "L2": 0.10128651, "L3": 0.10128651, "W": 0.12593918},
            {"i": 6, "L1": 0.10128651, "L2": 0.79742699, "L3": 0.10128651, "W": 0.12593918},
            {"i": 7, "L1": 0.10128651, "L2": 0.10128651, "L3": 0.79742699, "W": 0.12593918}
        ]
    }
    
    # Obter os dados de quadratura a partir do dicionário quadrature_table
    if (m, p) not in quadrature_table:
        raise ValueError(f"Regra de quadratura não definida para m={m}, p={p}.")

    # Recuperar os pontos e pesos da quadratura
    data = quadrature_table[(m, p)]
    points = [(entry['L2'], entry['L3']) for entry in data]  # Convertendo para (ξ, η)
    weights = [0.5 * entry['W'] for entry in data]
    
    return points, weights

# `triangle_rule()`

In [ ]:
def triangle_rule(ng):
    """
    Realiza a integração de uma função g(ξ, η) sobre o triângulo mestre utilizando quadratura de Gauss.
    
    Parâmetros:
    g : função - A função a ser integrada, deve aceitar dois argumentos (ξ, η).
    n : int - Número de pontos de quadratura (1, 3 ou 4 conforme a Tabela 5.9).
    
    Retorna:
    float - O valor aproximado da integral.
    """
    # Definição dos pontos e pesos de quadratura conforme a Tabela 5.9
    if ng == 1:
        points = [(1/3, 1/3)]
        weights = [0.5]
    elif ng == 3:
        points = [(0.5, 0), (0, 0.5), (0.5, 0.5)]
        weights = [1/6, 1/6, 1/6]
    elif ng == 4:
        points = [(1/3, 1/3), (0.6, 0.2), (0.2, 0.6), (0.2, 0.2)]
        weights = [-27/96, 25/96, 25/96, 25/96]
    elif ng == 6:
        points = [
            (0.44594849091597, 0.44594849091597),
            (0.44594849091597, 0.10810301816807),
            (0.10810301816807, 0.44594849091597),
            (0.09157621350977, 0.09157621350977),
            (0.09157621350977, 0.81684757298046),
            (0.81684757298046, 0.09157621350977),
        ]
        weights = [
            0.111690794839, 0.111690794839, 0.111690794839,
            0.054975871827, 0.054975871827, 0.054975871827,
        ]
    elif ng == 7:
        points = [
            (0.33333333333333, 0.33333333333333),
            (0.47014206410511, 0.47014206410511),
            (0.47014206410511, 0.05971587178977),
            (0.05971587178977, 0.47014206410511),
            (0.10128650732346, 0.10128650732346),
            (0.10128650732346, 0.79742698535309),
            (0.79742698535309, 0.10128650732346),
        ]
        weights = [
            0.225000000000, 0.132394152788, 0.132394152788,
            0.132394152788, 0.125939180544, 0.125939180544,
            0.125939180544,
        ]
    else:
        raise ValueError("Número de pontos de quadratura não suportado.")
    
    return (points, weights)

# `square_rule()`

In [ ]:
def square_rule(ng):
    """
    Realiza a integração de uma função g(ξ, η) sobre o quadrado mestre utilizando quadratura de Gauss.
    
    Parâmetros:
    g : função - A função a ser integrada, deve aceitar dois argumentos (ξ, η).
    n : int - Número de pontos de quadratura ao quadrado (1, 4 ou 9 conforme a Tabela 5.10).
    
    Retorna:
    float - O valor aproximado da integral.
    """
    # Definição dos pontos e pesos de quadratura conforme a Tabela 5.10
    if ng == 1:
        points = [(0, 0)]
        weights = [4]
    elif ng == 4:
        sqrt1_3 = 1 / np.sqrt(3)
        points = [(-sqrt1_3, -sqrt1_3), (sqrt1_3, -sqrt1_3),
                  (-sqrt1_3, sqrt1_3), (sqrt1_3, sqrt1_3)]
        weights = [1, 1, 1, 1]
    elif ng == 9:
        sqrt_06 = np.sqrt(0.6)
        points = [(-sqrt_06, -sqrt_06), (0, -sqrt_06), (sqrt_06, -sqrt_06),
                  (-sqrt_06, 0), (0, 0), (sqrt_06, 0),
                  (-sqrt_06, sqrt_06), (0, sqrt_06), (sqrt_06, sqrt_06)]
        weights = [25/81, 40/81, 25/81, 40/81, 64/81, 40/81, 25/81, 40/81, 25/81]
    else:
        raise ValueError("Número de pontos de quadratura não suportado. Escolha entre 1, 4 ou 9.")
    
    return (points, weights)   

# `quad_integrate()`

In [ ]:
def quad_integrate(f, rule):
    points, weights = rule
    # Cálculo da integral usando a fórmula de quadratura
    integral = 0
    for (xi, eta), w in zip(points, weights):
        integral += w * f(xi, eta)    
    return integral

# `gauss_data()`

In [ ]:
def gauss_data(FINITE_ELEMENT):
    # Desempacotar o tipo de elemento
    type, order = FINITE_ELEMENT

    # Get the Gauss points and weights
    if type == 'Triangle' and order in [1]:
        points, weights = general_triangle_rule(m=7, p=5)
    
    elif type == 'Triangle' and order in [2, 3]:
        points, weights = general_triangle_rule(m=7, p=5)
   
    elif type == 'Quadrangle':
        points, weights = square_rule(ng=9)

    return points, weights

In [ ]:
# # Exemplo de uso
# # Defina a função g(ξ, η) que deseja integrar, por exemplo:
# g = lambda xi, eta: xi ** 2 + eta ** 3 # Função exemplo

# # Calcule a integral usando 3 pontos de quadratura
# quad_triangle_3 = quad_integrate(g, triangle_rule(n=3))
# quad_square_4 = quad_integrate(g, square_rule(n=4))
# print("quad_triangle_3:", quad_triangle_3)
# print("quad_square_4:", quad_square_4)